In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [16]:
os.environ["KAGGLE_USERNAME"] = "amithsurasani"
os.environ["KAGGLE_KEY"] = "e366f530535dfa85184b503e42c7ce1c"

In [ ]:
name_of_dataset = "felipekitamura/spr-x-ray-age-and-gender-dataset"
download_path = "C:/Users/amith/Documents/GitHub/udemy/Age and Gender Prediction from Chest X-Ray Scans using CNN and OpenCV/data"
os.system(f"kaggle datasets download -d felipekitamura/spr-x-ray-age-and-gender-dataset")
print("Download over!")

In [14]:
os.listdir("C:/Users/amith/Documents/GitHub/udemy/Age and Gender Prediction from Chest X-Ray Scans using CNN and OpenCV")

['app', 'ML_model']